# TO DO
2. Now save the scrapped data into CSV and Sql lite Db ( Db name : MovieInfoDatabase Table name: MovieInfoTable) 
   Take user input and show the movie details asked for like director of the movie or the cast of the movie from movie_details
   varaible.
   - Satya  
   
4. Now after sorting the count and rating for the cast and director we should now give out analysis of their career.
   - Satya
    
   
6. write exception handelling if user input is wrong.(task4)
    - Satya
    
7. find cosine similarity between 5 different actors.
    - Manish.
    
* change all meta-score to meta_score.
* add important description above cells and delete unwanted cells.

   

# Tasks completed
1. Webscrapping of 500 movies and store them into dictionary
   - Manish
   
3. Now make a count of cast and director based on that give review ( we only did the cast part there is also director part)
     we store the cast & director data in varaibles : cast_details, director_details. 
   - Manish
   
5. Find Cosine similarities between directors by taking user input.
   - Manish 


#  Webscraping Module


1.import required libaries 
    * requests : To retrived the Html of the source page
    * bs4      : To converted the obtained html which in the form of text into soup object in which we can retrive specific                      data like headers, paragraphs, etc.
    * lxml     : Helps bs4 to understand the html.
    * csv      : To write or read into csv file.

In [2]:
import requests
import lxml
import bs4
import csv 
import time
from math import sqrt


2. Loading the featured page links into a link to hit them and creating headers to mimic actual browser.

In [3]:
urls =["https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=0",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=1",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=2",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=3",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=4"]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}



3. Initialise the varaible that are used to store the data

In [4]:
movie_short_href={} # used to store the href values 

movie_details={}    # Used to store the final movie details as 
                    #   { 'movie_name':{'Director':['director_name'], 
                    #                   'Principal Cast' : [name_1, name_2 ...name_m], 
                    #                    'Cast':['name_1, name_2... name_n'] }}  note m>n.
            
failed_items=[]     # Used to store the movie if of those movie where we have a failed attempt.




4. Below process is to achieve the link for each movies from the main page

    result : now we have collection of end points of the movies.

In [5]:

index_num=0
for url in urls:
    result = requests.get(url,headers=headers)
    soup = bs4.BeautifulSoup(result.text, "lxml")
    div_tag = soup.find_all('div', class_=["browse_list_wrapper one browse-list-large",
                                           "browse_list_wrapper two browse-list-large",
                                           "browse_list_wrapper three browse-list-large",
                                           "browse_list_wrapper four browse-list-large"])
    for item in div_tag:
        for element in item.find_all('td', class_='clamp-image-wrap'):
            index_num=index_num+1
            source=element.find_all('a' , href= True)[0]
            movie_short_href[index_num]=source['href'].replace('/movie/','')
    
    

5. This is method is to scrape individual movie details from its respective website, if there is any kind of failure while scraping we store the the movie id  in a list (failed_items).

In [6]:
def retrive_data_from_website(i):
    movie_link='https://www.metacritic.com/movie/'
    url = movie_link+movie_short_href[i]+'/details'
    result = requests.get(url,headers=headers)
    if result.status_code != 200:
        failed_items.append(i)
    else:
        
        final_details={}
        soup = bs4.BeautifulSoup(result.text,'lxml')
        tags = soup.find_all('table', class_='credits')
        wanted_columns=['Director', 'Principal Cast', 'Cast']
        movie_name = soup.find_all('div', class_='product_page_title oswald upper')[0].find_all('h1')[0].getText()
        meta_score = soup.find_all('a', class_ = 'metascore_anchor')[0].getText().strip()
        final_details['meta-score']=meta_score
        for table in tags:
            column = table.find_all('th', class_="person")[0].getText()
            if column in wanted_columns:
                details=[]
                for name in table.find_all('a', href=True):
                    details.append(name.getText().strip())
                final_details[column]=details
        if movie_name not in movie_short_href.values():
            movie_short_href[i] = movie_name
        movie_details[movie_short_href[i]] = final_details
    if i in [100,200,300,400,500]:
        time.sleep(2)
    if i in failed_items:
        failed_items.remove(i)
    

6. Now we hit every movie link from our collection and check for any failed attempts,
    if any there is any failed attempt, that implies that the href we obtained has different value which the website has
    internally rerouted, so just for the failed items instead of hitting the details  page we go step bye step and find the
    actual href value 
    
    example:
    movie href value = 'citizen-kane' in page - 1
    we would be hitting = '/movies/citizen-kane/details' which is wrong endpoint.
    actual endpoint = '/movies/citizen-kane-1941/details' 
    
   for movies like these which are less than 10 in count,
    we first hit the movie page which is '/movies/citizen-kane'  which would internal redirect at the server side into
   'movie/citizen-kane-1941' we now take the url and update the href value in movie_short_href variable and continue the 
    process
    

In [7]:

for i in movie_short_href.keys():
    retrive_data_from_website(i)
if len(failed_items)>0:
    for i in failed_items:
        url = 'https://www.metacritic.com/movie/'+movie_short_href[i]
        dummy = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
        movie_short_href[i]=dummy.url[dummy.url.rindex('/')+1::]
        retrive_data_from_website(i)


#  Store into CSV 


7. Now we use csv writed class to save our movie details into Csv file.

In [8]:
with open("dict.csv", "w", newline="") as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['Movie Name','Director','Cast','Meta Score'])
    for key, value in movie_details.items():
        writer.writerow([key,value['Director'],value.get('Principal Cast',[])+value.get('Cast',[]),value['meta-score']])


# Store in  SQL Lite Db
8. we store our movie details into sqlLite db under the name of MovieInfoDatabase db and into MovieInfoTable

In [9]:
import sqlite3
connection = sqlite3.connect('MovieInfoDatabase.db')
cursor= connection.cursor()
cursor.execute('CREATE TABLE MovieInfoTable (Movie_name Varchar, Director varchar, Cast Varchar, Meta_score varchar)')
for key, value in movie_details.items():
    cast=""
    for i in value.get('Principal Cast',[])+value.get('Cast',[]):
        cast =cast + i+','
    cast = cast[0:len(cast)-1:]
    sql_query="insert into MovieInfoTable values( \"{0}\",\"{1}\",\"{2}\",\"{3}\")".format(str(key), str(value['Director'][0]),str(cast),str(value['meta-score']))
    cursor.execute(sql_query)
connection.commit()

 9. Check for Movie info By giving movie name 
 

In [ ]:
movie_name = input('Which movie do you want to check? >')
desire_info = input('What information about this movie do you want to check? (Choose Director or Cast) >')
if movie_name not in movie_details.keys():
    print('The User provide movie name is not in our Data or there name doesn\'t match')
else:
    print(movie_details[movie_name][desire_info])


# Task 1:
1. Analyze how many times has each actor/actress appeared in these top 500 movies, analyze how many times has each director appeared in these top 500 movies, what can that tell you about their career?

We have accumulated all the details of cast and how many time that they have appeared in the in top 500 movies along with the movie rating provided by the metactritic website which is called as metascore. into the "cast_details" dictionary. 

Filter all the cast  and directors from each movie and append them to the list.
        cast_details = {'name' :{count:'v',rating:'v'}}
    director_details = {'name' :{count:'v',rating:'v'}}
                          
Count is the number of movies that they worked in.
Rating would be the average of the rating of the all the movies that they worked in.
    
                               
                        


In [97]:
cast_details={}

director_details={}

def add_castnames_count(cast_names,score):
    for name in cast_names:
        cast_details[name]= cast_details.get(name,{'count':0, 'rating': 0})
        cast_details[name]['count'] = cast_details[name].get('count',0) + 1
        cast_details[name]['rating'] = cast_details[name]['rating']+int(score)
            
def add_directornames_count(director_name,score):
    for name in director_name:
        director_details[name]= director_details.get(name,{'count':0, 'rating': 0})
        director_details[name]['count'] = director_details[name].get('count',0) + 1
        director_details[name]['rating'] = director_details[name]['rating']+int(score)   
        

for movie_name,movies in movie_details.items():
    add_castnames_count(list(set(movies.get('Principal Cast',[])+movies.get('Cast',[]))), movies['meta-score'])
    add_directornames_count(movies.get('Director',[]),movies['meta-score'])

# The code below is performed to find the average rating of the cast/director.
for name in cast_details.keys():
        cast_details[name]['rating'] = cast_details[name]['rating']/cast_details[name]['count']
for name in director_details.keys():
        director_details[name]['rating'] = director_details[name]['rating']/director_details[name]['count']
    


# # Task2 : Finding cosine similarities between  Directors

There are 410  distinct Director in the top 500 movies.

First we create a collection of  directors along with the cast they worked with and how many time that they have worked.

director_cast_details = {'director-1':{'cast-1' : 1, 'cast-2' : 3, ...}..}

 
Now for finding cosine similarity between two director we need find the all the cast which worked with director-1 and director-2 which we would call as union_cast, and then created a vector which consits of the number of time that they worked with the cast in the union_cast .
 
 suppose we take a pair of director as director-1, director-2 and lets look at the vector that we need to generate.
 
 Example:
 
 'director-1':{'cast-1' : 1, 
                'cast-2' : 2, }
 'director-2':{'cast-3' : 3, 
               'cast-2' : 4, }
               
  So the length of the vector is going to be the distinct cast of both directors.
  [cast-1, cast-2. cast-3]
  
  the director vectors are going to be 
  director-1= [1, 2, 0 ]
  
  director-2 = [0, 4, 3]
  
  now we give this input to our cosine function and get the similarity.

  
 
 

In [131]:
director_cast_details={}

for v in movie_details.values():
    for director_name in v['Director']:
        cast_data=director_cast_details.get(director_name,{})
        cast = list(set(v.get('Principal Cast',[])+(v.get('Cast',[]))))
        for name in cast:
            cast_data[name] = cast_data.get(name,0)+1
        director_cast_details[director_name]=cast_data

def inner_product(vector_1, vector_2):    #(1,2)(1,2)
    product_sum = 0
    for i in range(len(vector_1)):
        product_sum = product_sum +(vector_1[i]*vector_2[i])
    return product_sum
        
    
def cosine_similarity(director_1, director_2):
    union_cast= set(list(director_1.keys())+list(director_2.keys()))
    vector_1=[director_1.get(i,0) for i in union_cast]
    vector_2=[director_2.get(i,0) for i in union_cast]
    numerator = inner_product(vector_1,vector_2)
    if numerator==0:
        return 0
    denominator = sqrt(inner_product(vector_1,vector_1)) * sqrt(inner_product(vector_2,vector_2))
    if denominator==0:
        return 0
    return round(numerator/denominator,5)


while(True):
    print("Please choose from the below Options \n\t1.List out of all directors\n\t2.Cosine similarity between  directors \n\t3.Exit")
    choice= int(input())
    if choice==2:
        print("Please enter the name's of the director's you want to find the similarity")
        director_1 = str(input("Enter the name of First director."))
        director_2 = str(input("Enter the name of Second director."))
        print("The cosine similarity between {0} and {1} is {2}\n".format(director_1,director_2,cosine_similarity
                                                                        (director_cast_details[director_1],director_cast_details[director_2])))
    elif choice==3:
        print('Exit completed')
        break
    elif choice==1:
        print("Names of all director in top 500 movies")
        print(director_details.keys())
    

Please choose from the below Options 
	1.List out of all directors
	2.Cosine similarity between  directors 
	3.Exit
3
Exit completed


# Task 3 : Find similarity between 5 actors/actress

    1. Pick the top 5 cast  in terms of who have acted in most movies. And store them in cast_list
    2. Now we find the co_cast who worked with our cast and how many times did they work and store them in small_cast_details.
    3. Now we find cosine similarity between them and print them out.
    

In [136]:

my_dict={k: v for k, v in sorted(cast_details.items(), key=lambda item: item[1]['count'], reverse=True)}
names=iter(my_dict)
cast_list=[]
for i in range(5):
    cast_list.append(next(names))

small_cast_details={}
for name in cast_list:
    cast_data=small_cast_details.get(name,{})
    for k,v in movie_details.items():
        cast = list(set(v.get('Principal Cast',[])+(v.get('Cast',[]))))
        if name in cast:
            cast.remove(name)
            for cast_name in cast:
                cast_data[cast_name] = cast_data.get(cast_name,0)+1
            small_cast_details[name]=cast_data
            
for i in cast_list:
    cast_list.remove(i)
    for j in cast_list:
        print(i,j)
        print(cosine_similarity(small_cast_details[i],small_cast_details[j]))



John Ratzenberger Robert De Niro
0
John Ratzenberger Orson Welles
0
John Ratzenberger Marlon Brando
0
John Ratzenberger James Stewart
0
Orson Welles Robert De Niro
0
Orson Welles Marlon Brando
0
Orson Welles James Stewart
0
James Stewart Robert De Niro
0
James Stewart Marlon Brando
0


In [123]:
count=0
for k,v in movie_details.items():
    cast=list(set(v.get('Principal Cast',[])+v.get('Cast',[])))
    if 'John Ratzenberger' in cast:
        count=count+len(cast)-1
count

112

In [128]:
count=0
for i in small_cast_details['John Ratzenberger'].keys():
    count=count+small_cast_details['John Ratzenberger'][i]
count

112

In [125]:
{k: v for k, v in sorted(cast_details.items(), key=lambda item: item[1]['count'], reverse=True)}


{'John Ratzenberger': {'count': 7, 'rating': 92.57142857142857},
 'Robert De Niro': {'count': 7, 'rating': 91.85714285714286},
 'Orson Welles': {'count': 6, 'rating': 95.0},
 'Marlon Brando': {'count': 6, 'rating': 93.5},
 'James Stewart': {'count': 6, 'rating': 95.66666666666667},
 'Ward Bond': {'count': 6, 'rating': 91.66666666666667},
 'Harvey Keitel': {'count': 6, 'rating': 91.5},
 'Ralph Fiennes': {'count': 6, 'rating': 90.5},
 'Timothy Spall': {'count': 6, 'rating': 89.83333333333333},
 'Joseph Cotten': {'count': 5, 'rating': 94.2},
 'Robert Duvall': {'count': 5, 'rating': 92.8},
 'Al Pacino': {'count': 5, 'rating': 91.4},
 'Humphrey Bogart': {'count': 5, 'rating': 95.0},
 'John Qualen': {'count': 5, 'rating': 95.8},
 'Daniel Day-Lewis': {'count': 5, 'rating': 91.2},
 'Harrison Ford': {'count': 5, 'rating': 92.0},
 'Wallace Shawn': {'count': 5, 'rating': 91.8},
 'Max von Sydow': {'count': 5, 'rating': 88.8},
 'Willem Dafoe': {'count': 5, 'rating': 89.8},
 'Jim Broadbent': {'count

In [126]:
cast_list

['John Ratzenberger',
 'Robert De Niro',
 'Orson Welles',
 'Marlon Brando',
 'James Stewart']